In [2]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done


In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-11-01 05:18:51--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.41MB/s    in 0.2s    

2020-11-01 05:18:51 (5.41 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Furniture_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [7]:
# Recreating the vine table
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3VR960AHLFKDV|          4|            0|          0|   N|                Y|
|R16LGVMFKIUT0G|          5|            0|          0|   N|                Y|
|R1AIMEEPYHMOE4|          5|            1|          1|   N|                Y|
|R1892CCSZWZ9SR|          3|            0|          0|   N|                Y|
|R285P679YWVKD1|          3|            0|          0|   N|                N|
| RLB33HJBXHZHU|          5|            0|          0|   N|                Y|
|R1VGTZ94DBAD6A|          5|            2|          2|   N|                Y|
|R168KF82ICSOHD|          5|            0|          0|   N|                Y|
|R20DIYIJ0OCMOG|          5|            0|          0|   N|                Y|
| RD46RNVOHNZSC|          5|            0|          0|   N|     

In [29]:
# Total count of data
vine_df.count()

792113

In [9]:
# Creating new df with total votes more than 20
high_votes_vine_df = vine_df.filter("total_votes >= 20")
high_votes_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|
| RR99CPG695T0I|          5|          215|        248|   N|                N|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y|
|R3GNSIFV1J2Y2B|          1|           15|         60|   N|                N|
| RTCRZARYY4LXX|          5|           52|         54|   N|                Y|
|R3OFB4P7Y8WR27|          1|           15|         26|   N|                Y|
|R3MTAYGQM25N63|          4|           58|         59|   N|     

In [11]:
# Creating new df with helpful_votes/total_votes >= 50%
higher_percent_helpful_votes_df = high_votes_vine_df.withColumn("greater_than_fifty_percent", high_votes_vine_df["helpful_votes"] / high_votes_vine_df["total_votes"]).filter("greater_than_fifty_percent >= 0.5")
higher_percent_helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+--------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|greater_than_fifty_percent|
+--------------+-----------+-------------+-----------+----+-----------------+--------------------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|        0.9212121212121213|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|        0.9130434782608695|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|                       1.0|
| RR99CPG695T0I|          5|          215|        248|   N|                N|        0.8669354838709677|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|        0.9772727272727273|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y|        0.9615384615384616|
| RTCRZARYY4LXX|          5|           52|         54| 

In [12]:
# creating new df where vine equals "Y"
Y_vine_df = higher_percent_helpful_votes_df.filter(higher_percent_helpful_votes_df["vine"] == "Y")
Y_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+--------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|greater_than_fifty_percent|
+--------------+-----------+-------------+-----------+----+-----------------+--------------------------+
|R2BQOD1R0228FN|          3|           17|         26|   Y|                N|        0.6538461538461539|
| RC31RUPFOHBHQ|          4|          102|        117|   Y|                N|        0.8717948717948718|
| REN3N1WITLF1Y|          5|           33|         37|   Y|                N|        0.8918918918918919|
| R71RZQ9UZVG47|          4|           38|         47|   Y|                N|        0.8085106382978723|
|R38NMQBH88HLM6|          4|           18|         24|   Y|                N|                      0.75|
|R33FGX9EE3QVR6|          4|           26|         26|   Y|                N|                       1.0|
|R1KIOIK6WEYE59|          3|           19|         20| 

In [13]:
# creating new df where vine equals "N"
N_vine_df = higher_percent_helpful_votes_df.filter(higher_percent_helpful_votes_df["vine"] == "N")
N_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+--------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|greater_than_fifty_percent|
+--------------+-----------+-------------+-----------+----+-----------------+--------------------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|        0.9212121212121213|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|        0.9130434782608695|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|                       1.0|
| RR99CPG695T0I|          5|          215|        248|   N|                N|        0.8669354838709677|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|        0.9772727272727273|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y|        0.9615384615384616|
| RTCRZARYY4LXX|          5|           52|         54| 

In [22]:
# Calculating total number of reviews when vine equals "Y"
Y_vine_total_reviews = Y_vine_df.agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "total_vine_equals_Y_reviews")
Y_vine_total_reviews.show()


+---------------------------+
|total_vine_equals_Y_reviews|
+---------------------------+
|                        136|
+---------------------------+



In [23]:
# Calculating total number of 5-Star reviews when vine equals "Y"
Y_vine_5star_reviews = Y_vine_df.filter("star_rating == 5").agg({"star_rating":"count"}).withColumnRenamed("count(star_rating)", "total_5star_Y_vine_reviews")
Y_vine_5star_reviews.show()

+--------------------------+
|total_5star_Y_vine_reviews|
+--------------------------+
|                        74|
+--------------------------+



In [24]:
# Calculating percentage of 5-Star reviews when vine equals "Y"
Y_vine_5star_percent = Y_vine_5star_reviews.collect()[0]["total_5star_Y_vine_reviews"] / Y_vine_total_reviews.collect()[0]["total_vine_equals_Y_reviews"] * 100
Y_vine_5star_percent


54.41176470588235

In [25]:
# Calculating total number of reviews when vine equals "N"
N_vine_total_reviews = N_vine_df.agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "total_vine_equals_N_reviews")
N_vine_total_reviews.show()

+---------------------------+
|total_vine_equals_N_reviews|
+---------------------------+
|                      18019|
+---------------------------+



In [26]:
# Calculating total number of 5-Star reviews when vine equals "N"
N_vine_5star_reviews = N_vine_df.filter("star_rating == 5").agg({"star_rating":"count"}).withColumnRenamed("count(star_rating)", "total_5star_N_vine_reviews")
N_vine_5star_reviews.show()

+--------------------------+
|total_5star_N_vine_reviews|
+--------------------------+
|                      8482|
+--------------------------+



In [27]:
# Calculating percentage of 5-Star reviews when vine equals "N"
N_vine_5star_percent = N_vine_5star_reviews.collect()[0]["total_5star_N_vine_reviews"] / N_vine_total_reviews.collect()[0]["total_vine_equals_N_reviews"] * 100
N_vine_5star_percent


47.072534546867196